In [11]:
import requests
import re
import io
import pandas as pd

def finstate_summary(code: str):
    '''
    요약 재무제표 데이터를 가져와 CSV 문자열을 반환합니다
    :param code: 종목코드
    '''
    print(f'finstate_summary({code}) calling') # 호출 확인을 위한 print

    # fin_type: 재무제표 종류 '0'=주재무제표, '1'=K-GAAP개별, '2'=K-GAAP연결, '3'=K-IFRS별도, '4'=K-IFRS연결
    # freq: 기간: Y=년(기본), Q=분기, 'A'=연간분기 전체
    fin_type, freq ='0', 'Y'

    # encparam 읽어오기
    url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930'
    html_text = requests.get(url).text

    if not re.search("encparam: '(.*?)'", html_text):
        print('encparam not found') # encparam이 없는 경우
        return None
    encparam = re.findall ("encparam: '(.*?)'", html_text)[0]

    url = f'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={code}&fin_typ={fin_type}&freq_typ={freq}&encparam={encparam}'
    r = requests.get(url, headers={'Referer': url})
    df_list = pd.read_html(io.StringIO(r.text), encoding='euc-kr')
    df = df_list[1]
    df.columns = [col[1] for col in df.columns]
    df.set_index('주요재무정보', inplace=True)
    df.columns = [re.sub('[^\.\d]', '', col) for col in df.columns]
    df.columns = [pd.to_datetime(col, format='%Y%m', errors='coerce') for col in df.columns]
    df = df.transpose()
    df.index.name = '날짜'
    return df.to_csv() ## CSV 문자열을 반환

df = finstate_summary('017670') # SK텔레콤(017670)
df


finstate_summary(017670) calling


'날짜,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,CAPEX,FCF,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)\n2018-12-01,168740.0,12018.0,12018.0,39760.0,31320.0,31279.0,41.0,423691.0,200199.0,223492.0,224708.0,-1216.0,446.0,43326.0,-40477.0,-2383.0,27924.0,15402.0,100761.0,7.12,18.56,15.52,8.26,89.58,56140.21,7748.0,6.96,62532.0,0.86,2000.0,3.71,22.94,403728555.0\n2019-12-01,177407.0,11082.0,11082.0,11610.0,8607.0,8887.0,-280.0,452024.0,223854.0,228169.0,229502.0,-1333.0,446.0,40350.0,-35816.0,-6867.0,33759.0,6591.0,119788.0,6.25,4.85,3.91,1.97,98.11,56328.7,2201.0,21.62,62760.0,0.76,2000.0,4.2,82.15,403728555.0\n2020-12-01,160877.0,12486.0,12486.0,9052.0,15005.0,15044.0,-38.0,479070.0,235107.0,243962.0,237439.0,6523.0,446.0,58219.0,-42504.0,-14576.0,35578.0,22641.0,125800.0,7.76,9.33,6.44,3.22,96.37,58016.08,3726.0,12.77,66577.0,0.71,2000.0,4.2,47.53,

In [8]:
df = finstate_summary('017670') # SK텔레콤(017670)
df

finstate_summary(017670) calling


'날짜,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,CAPEX,FCF,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)\n2018-12-01,168740.0,12018.0,12018.0,39760.0,31320.0,31279.0,41.0,423691.0,200199.0,223492.0,224708.0,-1216.0,446.0,43326.0,-40477.0,-2383.0,27924.0,15402.0,100761.0,7.12,18.56,15.52,8.26,89.58,56140.21,7748.0,6.96,62532.0,0.86,2000.0,3.71,22.94,403728555.0\n2019-12-01,177407.0,11082.0,11082.0,11610.0,8607.0,8887.0,-280.0,452024.0,223854.0,228169.0,229502.0,-1333.0,446.0,40350.0,-35816.0,-6867.0,33759.0,6591.0,119788.0,6.25,4.85,3.91,1.97,98.11,56328.7,2201.0,21.62,62760.0,0.76,2000.0,4.2,82.15,403728555.0\n2020-12-01,160877.0,12486.0,12486.0,9052.0,15005.0,15044.0,-38.0,479070.0,235107.0,243962.0,237439.0,6523.0,446.0,58219.0,-42504.0,-14576.0,35578.0,22641.0,125800.0,7.76,9.33,6.44,3.22,96.37,58016.08,3726.0,12.77,66577.0,0.71,2000.0,4.2,47.53,

In [12]:
def stock_history(code: str):
    print(f'stock_history({code}) calling')
    url = f'https://fchart.stock.naver.com/sise.nhn?symbol={code}&timeframe=day&count=1000&requestType=0'
    r = requests.get(url)
    data = r.text.split('\n')
    data = [d.split('|') for d in data if d]
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df.set_index('date', inplace=True)
    df = df.astype(float)
    return df

df = stock_history('005930') # 삼성전자(005930)
df



stock_history(005930) calling


ValueError: non convertible value <?xml version="1.0" encoding="EUC-KR" ?> with the unit 'ms', at position 0